In [5]:
import pymongo
import json

In [6]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["twitter"]

In [77]:
db.tweets.drop()

In [80]:
db.tweets.count_documents({})

0

In [ ]:
selected_columns = ['id', 'created_at', 'text', 'source', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities']

def filter_func(item):
    key, value = item
    
    if key in selected_columns:
        return True
    else:
        return False

In [ ]:
def cleaning(data):
        doc = dict(filter(filter_func, data.items()))
        doc['user_id'] = data['user']['id']
        doc['hashtags'] = doc['entities']['hashtags']
        doc['user_mentions'] = doc['entities']['user_mentions']
        doc['_id'] = doc['id']
        del doc['id']
        del doc['entities']
        return doc

In [ ]:
with open("corona-out-sample", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            doc = cleaning(data)
                        
            if 'retweeted_status' in data:
                doc['retweeted_status_id'] = data['retweeted_status']['id']
                retweet = cleaning(data['retweeted_status'])

                try:
                    db.tweets.insert_one(retweet)
                except Exception:
                    pass
                    
            db.tweets.insert_one(doc)
            
        except Exception:
            continue